In [12]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', shuffle=True)

In [27]:
Prices_with_changes = pd.read_csv('../DataFiles/withPricesAndChange.csv')


Prices_with_changes=  Prices_with_changes.round(4)
Prices_with_changes.loc[(Prices_with_changes.StockChg == 'Positive'), 'StockChg'] = 1
Prices_with_changes.loc[(Prices_with_changes.StockChg == 'Negative'), 'StockChg'] = 0
Prices_with_changes["StockChg"] = pd.to_numeric(Prices_with_changes["StockChg"])

Prices_with_changes

,title,Symbol,Security,Year,Month,Day,Date,Key,Opening,Closing,StockChg,WeekNo
0,Agilent to Buy Multiplicom for ‚Ç¨68M in Cash,A,Agilent Technologies,2016,12,20,2016-12-20,2016-12-20A,45.75,46.21,1,52
1,Pressure Biosciences CEO On Its Beaten Down St...,A,Agilent Technologies,2016,12,6,2016-12-06,2016-12-06A,44.58,44.84,1,50
2,"The Market In 5 Minutes: DryShips, Trump And T...",A,Agilent Technologies,2016,11,16,2016-11-16,2016-11-16A,46.30,46.18,0,47
3,A Peek Into The Markets: U.S. Stock Futures De...,A,Agilent Technologies,2016,11,16,2016-11-16,2016-11-16A,46.30,46.18,0,47
4,"Keep an Eye on These 8 Stocks for November 16,...",A,Agilent Technologies,2016,11,16,2016-11-16,2016-11-16A,46.30,46.18,0,47
...,...,...,...,...,...,...,...,...,...,...,...,...
176894,Mid-Morning Market Update: Unemployment Reache...,ZTS,Zoetis,2013,2,1,2013-02-01,2013-02-01ZTS,31.50,31.01,0,5
176895,Pfizer Offers Statement on Zoetis IPO,ZTS,Zoetis,2013,2,1,2013-02-01,2013-02-01ZTS,31.50,31.01,0,5
176896,"Zoetis IPO Opens at $31.50/Share, Priced at $26",ZTS,Zoetis,2013,2,1,2013-02-01,2013-02-01ZTS,31.50,31.01,0,5
176897,Zoetis IPO to Open Shortly,ZTS,Zoetis,2013,2,1,2013-02-01,2013-02-01ZTS,31.50,31.01,0,5


In [28]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()

big_ol_text_blob = Prices_with_changes['title']

big_ol_text_blob = big_ol_text_blob.tolist()
big_ol_text_blob = big_ol_text_blob
#big_ol_text_blob = ''.join(big_ol_text_blob)

#big_ol_text_blob = [big_ol_text_blob]


counts = cv.fit_transform(big_ol_text_blob)
counts.shape

(176899, 36953)

In [29]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X = tfidf_transformer.fit_transform(counts) 
y = np.array(Prices_with_changes['StockChg'])

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0)

In [32]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB().fit(x_train, y_train)

predictions = model.predict(x_test)
predictions


array([1, 0, 1, ..., 1, 1, 1], dtype=int64)

In [33]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))


              precision    recall  f1-score   support

           0       0.54      0.37      0.44     16942
           1       0.55      0.70      0.62     18438

    accuracy                           0.55     35380
   macro avg       0.54      0.54      0.53     35380
weighted avg       0.54      0.55      0.53     35380



In [ ]:
#use stddev to determine if the movement is statictically signimifant. need historic stock data for the day each of these various articles was written. 